# Anticipez les besoins en consommation électrique de bâtiments

- **Projet 4 du parcours « Data Scientist » d’OpenClassrooms**
- **Mark Creasey**

## Partie 2 : Modélisation et Résultats

<img height="64" src="https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png" alt="Logo seattle">


## 1.1 Mission

À partir des [relevés déjà réalisés en 2015 et 2016](https://www.kaggle.com/city-of-seattle/sea-building-energy-benchmarking#2015-building-energy-benchmarking.csv) :

- **prédire les émissions de CO2** et la **consommation totale d’énergie** de bâtiments commerciales en Seattle pour lesquels elles n’ont pas encore été mesurées, basé sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction ...).

- **évaluer l’intérêt de l’[ENERGY STAR Score](https://www.energystar.gov/buildings/facility-owners-and-managers/existing-buildings/use-portfolio-manager/interpret-your-results/what) pour la prédiction d’émissions**.


## 1.2 Requirements : Bibliothèques utilisées dans ce notebook


### 1.2.1 Import des bibliothèques


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import sklearn


### 1.2.2 Liste des versions des bibliothèques utilisées


In [2]:
from platform import python_version
python_version()

print('versions des bibliothèques utilisées:')
print('; '.join(f'{m.__name__}=={m.__version__}' for m in globals(
).values() if getattr(m, '__version__', None)))


versions des bibliothèques utilisées:
numpy==1.21.5; pandas==1.1.5; seaborn==0.11.2; scipy==1.7.3; sklearn==1.0.2


### 1.2.3 Configuration défauts d'affichage


In [3]:
pd.set_option('display.max_columns', 200)  # pour afficher toutes les colonnes
pd.set_option('display.max_rows', 20)  # pour afficher max 10 lignes
pd.set_option('display.max_colwidth', 800)  # pour afficher toutes la text
# pd.set_option('display.precision', 2)

%matplotlib inline
sns.set_theme(style="white", context="notebook")
sns.set_color_codes("pastel")
sns.set_palette("tab20")


## 1.3 Des fonctions utilitaires

Pour enregistrer les graphiques, define **`SAVE_IMAGES = True`**


In [4]:
SAVE_IMAGES = True
IMAGE_FOLDER = './images'
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)


def to_png(fig_name=None):
    """
    Enregistre l'image dans un fichier,
    il faut appeler avant plt.show() pour pouvoir ajuster la taille de l'image
    avec bbox_inches=tight pour être sûr d'inclure le titre / legend entier.
    """
    def get_title():
        if plt.gcf()._suptitle is None:  # noqa
            return plt.gca().get_title()
        else:
            return plt.gcf()._suptitle.get_text()  # noqa

    if SAVE_IMAGES:
        if fig_name is None:
            fig_name = get_title()
        elif len(fig_name) < 9:
            fig_name = f'{fig_name}_{get_title()}'
        fig_name = fig_name.replace(' ', '_').replace(
            ':', '-').replace('.', '-').replace('/', '_')
        print(f'"{fig_name}.png"')
        plt.gcf().savefig(
            f'{IMAGE_FOLDER}/{fig_name}.png', bbox_inches='tight')


### 1.3.2 Vérifier que les colonnes sont dans le dataframe

- sans changer l'ordre des colonnes


In [5]:
def cols_in_df(df: pd.DataFrame, colonnes: list = None) -> list:
    """Procedure pour retourner les colonnes existantes dans le dataframe dans la même ordre.
    Utiliser pour assurer que les colonnes existe.
    """
    ret_cols = []
    for col in colonnes:
        if col in df.columns:
            ret_cols.append(col)
    return ret_cols


## 1.4 Les métriques d'évaluation


# 2. Données d'entrainement et test


## Import des données nettoyées


In [6]:
model_df = pd.read_csv('data/out/cleaned_data.csv')
model_df.head()


,OSEBuildingID,DataYear,NumberofFloors,LargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEnergyUse(kBtu),TotalGHGEmissions,ZipCode,PrimaryUseType,building_age,area_per_floor,percent_parking,distance(km)
0,1,2015,12.0,88434.0,65.0,6981428.0,249.43,98101,hotel-like,95,7369.500000,0.000000,0.556751
1,2,2015,11.0,83880.0,51.0,8354235.0,263.51,98101,hotel-like,26,8045.636364,0.145453,0.354915
2,3,2015,41.0,757243.0,18.0,73130656.0,2061.48,98101,hotel-like,53,23463.170732,0.000000,0.442002
3,5,2015,10.0,61320.0,1.0,28229320.0,1936.34,98101,hotel-like,96,6132.000000,0.000000,0.315723
4,8,2015,18.0,123445.0,67.0,14829099.0,507.70,98121,hotel-like,42,5968.333333,0.103929,0.608448


### Vérification que toutes les colonnes sont 100% remplies


In [7]:
model_df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2986 entries, 0 to 2985
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OSEBuildingID              2986 non-null   int64  
 1   DataYear                   2986 non-null   int64  
 2   NumberofFloors             2986 non-null   float64
 3   LargestPropertyUseTypeGFA  2986 non-null   float64
 4   ENERGYSTARScore            1990 non-null   float64
 5   SiteEnergyUse(kBtu)        2986 non-null   float64
 6   TotalGHGEmissions          2986 non-null   float64
 7   ZipCode                    2986 non-null   int64  
 8   PrimaryUseType             2986 non-null   object 
 9   building_age               2986 non-null   int64  
 10  area_per_floor             2986 non-null   float64
 11  percent_parking            2986 non-null   float64
 12  distance(km)               2986 non-null   float64
dtypes: float64(8), int64(4), object(1)
memory usage:

In [8]:
model_df['ENERGYSTARScore'].isna().sum()/len(model_df)


0.333556597454789

On voit que toutes les colonnes sont 100% non_null, sauf ENERGYSTARScore (1990 de 2986 valeurs remplis)

L'ENERGYSTARScore est applicable seulement pour 21 des plus de 80 PropertyTypes, donc cette colonne n'est pas 100% remplie. Pour la première partie de la modélisation, on n'utilise pas cette colonne pour les prévisions.

- Imputer l'ENERGYSTARScore pour les autres PropertyTypes n'a aucun sense, car c'est un ranking parmi les bâtiments de la même PropertyType dans tout le pays

- On évaluera plus tard l'utilité de cette colonne seulement pour les PropertyTypes ou il est présent, en excluant les lignes ou ce n'est pas présent


### Corriger datatypes

Certains champs numériques sont catégoriques (ZipCode, DataYear), ou identifiants des bâtiments (OSEBuildingID) :

- **DataYear** et **ZipCode**ZipCode sont potentiellement des variables catégoriques à inclure dans les modèles
- **OSEBuildingID** sera utilisé pour analyser et interpréter les erreurs des prédictions, mais ce champ n'est pas utilisé dans la modélisation

On les convertit en type 'object'


In [9]:
object_cols = ['DataYear', 'ZipCode', 'OSEBuildingID']


def corriger_datatypes(df, obj_cols=object_cols):
    df = df.copy()
    for col in obj_cols:
        if col in df:
            df[col] = df[col].astype('object')
    return df


model_df = model_df.pipe(corriger_datatypes)
model_df.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2986 entries, 0 to 2985
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   OSEBuildingID              2986 non-null   object 
 1   DataYear                   2986 non-null   object 
 2   NumberofFloors             2986 non-null   float64
 3   LargestPropertyUseTypeGFA  2986 non-null   float64
 4   ENERGYSTARScore            1990 non-null   float64
 5   SiteEnergyUse(kBtu)        2986 non-null   float64
 6   TotalGHGEmissions          2986 non-null   float64
 7   ZipCode                    2986 non-null   object 
 8   PrimaryUseType             2986 non-null   object 
 9   building_age               2986 non-null   int64  
 10  area_per_floor             2986 non-null   float64
 11  percent_parking            2986 non-null   float64
 12  distance(km)               2986 non-null   float64
dtypes: float64(8), int64(1), object(4)
memory usage:

## Choix de variables independents (X) et la variable à prédire (y)

On va créer plusieurs modèles prenant compte de différents variables independents, pour prédire soit consommation énergétique, soit les emissions CO2. Ci-dessous, une fonction pour simplifier le tâche


In [10]:
print(model_df.columns.tolist())


['OSEBuildingID', 'DataYear', 'NumberofFloors', 'LargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SiteEnergyUse(kBtu)', 'TotalGHGEmissions', 'ZipCode', 'PrimaryUseType', 'building_age', 'area_per_floor', 'percent_parking', 'distance(km)']


In [11]:
colonnes_x = ['DataYear', 'NumberofFloors', 'LargestPropertyUseTypeGFA', 'ZipCode',
              'PrimaryUseType', 'building_age', 'area_per_floor', 'percent_parking', 'distance(km)']

colonnes_cibles = ['SiteEnergyUse(kBtu)', 'TotalGHGEmissions']


def create_x_y(df, cols_x=colonnes_x, col_y=['SiteEnergyUse(kBtu)']):
    cols_x = cols_in_df(df, cols_x)
    print(f'create_x_y, y={col_y}, x={cols_x}')
    return df[cols_x], df[col_y]


bldg_id = model_df[['OSEBuildingID']]
x, y = model_df.pipe(create_x_y)
print(x.shape)
print(y.shape)


create_x_y, y=['SiteEnergyUse(kBtu)'], x=['DataYear', 'NumberofFloors', 'LargestPropertyUseTypeGFA', 'ZipCode', 'PrimaryUseType', 'building_age', 'area_per_floor', 'percent_parking', 'distance(km)']
(2986, 9)
(2986, 1)


# 3. Prédiction de la consommation énergétique


## Modèle baseline (dummyRegressor)


## Modèle linéaire


## Modèle non-linéaire


## Modèle ensembliste


## Évaluation des prédictions (analyse d'erreurs)


## Comparaison et choix du modèle


# 4. Prédiction des émissions CO2


# 5. Influence de l'ENERGYSTARScore sur les prédictions


# 6. Résultats

## Comparaison des performances des modèles


# 7. Conclusion
